In [2]:
# Importing the necessary dependencies
import pandas as pd
import numpy as np
from copy import deepcopy
from string import punctuation
from random import shuffle

import gensim
from gensim.models.word2vec import Word2Vec
LabeledSentence = gensim.models.doc2vec.LabeledSentence

from keras.utils import np_utils

from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
import pickle
from nltk.tokenize import TweetTokenizer
tokenizer = TweetTokenizer()
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Dense, Activation, LSTM, Embedding
from sklearn.model_selection import train_test_split

In [4]:
# get the data
# Convert the labels from str to int.
def getData():
    data = pd.read_csv("dataset/mydata.csv",error_bad_lines=False, encoding = "ISO-8859-1",header=None)
    data.columns = ["SentimentText","Sentiment"]
    data['Sentiment'] = data['Sentiment'].map(int)
    return data

data = getData()
pos = data[data.Sentiment == 1]
neg = data[data.Sentiment == 0]
print("Pos examples:",pos.shape,"Neg examples:", neg.shape)
data.head()

Pos examples: (27374, 2) Neg examples: (8344, 2)


,SentimentText,Sentiment
0,RT @Mayank_M_Joshi: Kovind will make an except...,1
1,@narendramodi @ashrafghani Hii pm modi good mo...,1
2,"And this "" love for Dalit "" emerged to sidelin...",1
3,RT @aijazzakasyed: @AijazZakaSyed writes in @S...,1
4,@soniandtv @vivekagnihotri Mamatajiððð...,1


In [29]:
# Tokenize the tweets
# Remove the tokens which start with '@' or 'http' or '#
slangs = pickle.load(open("dataset/slang.pkl","rb"))
def tokenize(tweet):
    try:
        tweet = str(tweet).lower()
        tokens = tokenizer.tokenize(tweet)
        tokens = [ token for token in tokens if not ( token.startswith("@") or token.startswith("http") or token.startswith("#")) ]
        for i in range(len(tokens)):
            try:
                tokens[i] = slangs[tokens[i]]
            except:
                continue
        return tokens
    except:
        return None

In [30]:
def postprocess(data):
    data['tokens'] = data['SentimentText'].progress_map(tokenize)
    return data

data = postprocess(data)

progress-bar: 100%|██████████| 35718/35718 [00:02<00:00, 16310.75it/s]


In [31]:
# Break the data into test train split
n = data.shape[0]
x_train, x_test, y_train, y_test = train_test_split(np.array(data.head(n).tokens),
                                                    np.array(data.head(n).Sentiment), test_size=0.2)


In [32]:
def labelizeTweets(tweets, label_type):
    labelized = []
    for i,v in tqdm(enumerate(tweets)):
        label = '%s_%s'%(label_type,i)
        labelized.append(LabeledSentence(v, [label]))
    return labelized

x_train = labelizeTweets(x_train, 'TRAIN')
x_test = labelizeTweets(x_test, 'TEST')

28574it [00:00, 501962.41it/s]
7144it [00:00, 494702.13it/s]


In [33]:
# Load the google's trained word2vec model
tweet_w2v = gensim.models.KeyedVectors.load_word2vec_format("dataset/google.bin.gz", binary=True)

In [34]:
# Converting a scentence to vector
# Basically averaging the vectors of different words
def wordvector(tokens, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in tokens:
        try:
            vec += tweet_w2v[word].reshape((1, size))
            count += 1.
        except KeyError:
            continue
    if count != 0:
        vec /= count
    return vec

In [35]:
from sklearn.preprocessing import scale
n_dim = 300
train_vecs_w2v = np.concatenate([wordvector(z, n_dim) for z in tqdm(map(lambda x: x.words, x_train))])
train_vecs_w2v = scale(train_vecs_w2v)

test_vecs_w2v = np.concatenate([wordvector(z, n_dim) for z in tqdm(map(lambda x: x.words, x_test))])
test_vecs_w2v = scale(test_vecs_w2v)

28574it [00:01, 17657.75it/s]
7144it [00:00, 18503.20it/s]


In [36]:
# Converting labels to one hot vector
# 0 -> [0,1]
# 1 - [1,0]
train_y = np_utils.to_categorical(y_train)
test_y = np_utils.to_categorical(y_test)

In [37]:
# building a model with 2 hidden layers
model = Sequential()
model.add(Dense(512, activation='relu', input_dim=300, kernel_initializer='normal'))
model.add(Dense(512, activation='relu', kernel_initializer='normal'))
model.add(Dense(2, activation='softmax', kernel_initializer='normal'))
model.compile(optimizer=Adam(lr=0.0001),
              loss='mean_squared_error',
              metrics=['accuracy'])
model.fit(train_vecs_w2v, train_y, validation_data=(test_vecs_w2v, test_y), epochs=30, batch_size=100, verbose=1)

Train on 28574 samples, validate on 7144 samples
Epoch 1/30
28574/28574 [==============================] - 2s - loss: 0.0484 - acc: 0.9488 - val_loss: 0.0118 - val_acc: 0.9894
Epoch 2/30
28574/28574 [==============================] - 1s - loss: 0.0066 - acc: 0.9950 - val_loss: 0.0056 - val_acc: 0.9943
Epoch 3/30
28574/28574 [==============================] - 2s - loss: 0.0029 - acc: 0.9975 - val_loss: 0.0013 - val_acc: 0.9987
Epoch 4/30
28574/28574 [==============================] - 1s - loss: 0.0013 - acc: 0.9990 - val_loss: 6.1183e-04 - val_acc: 0.9993
Epoch 5/30
28574/28574 [==============================] - 1s - loss: 5.4864e-04 - acc: 0.9997 - val_loss: 4.2846e-04 - val_acc: 0.9996
Epoch 6/30
28574/28574 [==============================] - 1s - loss: 3.0154e-04 - acc: 0.9999 - val_loss: 3.8725e-04 - val_acc: 0.9996
Epoch 7/30
28574/28574 [==============================] - 1s - loss: 2.3587e-04 - acc: 0.9999 - val_loss: 3.3796e-04 - val_acc: 0.9996
Epoch 8/30
28574/28574 [==========

In [39]:
m = 108
print(data.SentimentText[m], data.Sentiment[m])
scentence = data.tokens[m]
scentence = tokenize("Kovind will make an exceptional president: PM Narendra Modi")
vec = wordvector(scentence, 300)
print(model.predict(vec))

To make the #InternationalYogaDay programme in #Lucknow a success, various NGOs and #Yoga gurus have camped there. https://t.co/TNuAucVF2Q 1
[[ 0.31715828  0.68284172]]


In [54]:
model.save("model1.h5")